In [ ]:
!pip install tensorflow scikit-optimize


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 2.4 MB/s eta 0:00:00


In [ ]:
# Import necessary libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import StratifiedKFold
from skopt import gp_minimize
from skopt.space import Real
import matplotlib.pyplot as plt

# Load CIFAR-10 Dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize the images
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
# Define Adaptive Focal Loss (AFL)
def adaptive_focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss(y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1 - 1e-7)  # Avoid log(0)
        ce_loss = -y_true * tf.math.log(y_pred)
        modulating_factor = tf.pow(1 - y_pred, gamma)
        loss = alpha * modulating_factor * ce_loss
        return tf.reduce_sum(loss, axis=-1)
    return focal_loss


In [ ]:
# Define the CNN Model
def create_model(gamma=2.0, alpha=0.25, optimizer='adam'):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam', loss=adaptive_focal_loss(gamma, alpha), metrics=['accuracy'])

    return model

In [ ]:
# Perform Cross-Validation with Stratified K-Fold
def cross_validate_model(x_data, y_data, n_splits=5, optimizer='adam', gamma=2.0, alpha=0.25):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    fold_accuracies = []

    for train_idx, val_idx in skf.split(x_data, np.argmax(y_data, axis=1)):
        x_train_cv, x_val_cv = x_data[train_idx], x_data[val_idx]
        y_train_cv, y_val_cv = y_data[train_idx], y_data[val_idx]

        model = create_model(gamma=gamma, alpha=alpha, optimizer=optimizer)
        model.fit(x_train_cv, y_train_cv, epochs=5, batch_size=64, verbose=0, validation_data=(x_val_cv, y_val_cv))

        _, val_acc = model.evaluate(x_val_cv, y_val_cv, verbose=0)
        fold_accuracies.append(val_acc)

    mean_acc = np.mean(fold_accuracies)
    std_acc = np.std(fold_accuracies)
    return mean_acc, std_acc

In [ ]:
# Train the model with Normal AFL
# By default gamma = 2.0 and alpha = 0.25
normal_model = create_model(gamma=2.0, alpha=0.25)

# Train for 5 epochs
history_normal = normal_model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_test, y_test), verbose=1)

# Evaluate on test data
normal_loss, normal_acc = normal_model.evaluate(x_test, y_test, verbose=0)
print(f"Normal AFL Test Accuracy: {normal_acc:.4f}")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 74s 92ms/step - accuracy: 0.3702 - loss: 0.3130 - val_accuracy: 0.5477 - val_loss: 0.2036
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 79s 89ms/step - accuracy: 0.5779 - loss: 0.1893 - val_accuracy: 0.5821 - val_loss: 0.1825
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 86s 95ms/step - accuracy: 0.6390 - loss: 0.1557 - val_accuracy: 0.6311 - val_loss: 0.1620
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 69s 88ms/step - accuracy: 0.6789 - loss: 0.1348 - val_accuracy: 0.6502 - val_loss: 0.1490
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 84s 90ms/step - accuracy: 0.7044 - loss: 0.1201 - val_accuracy: 0.6760 - val_loss: 0.1380
Normal AFL Test Accuracy: 0.6760


In [ ]:
# Define the parameter search space for gamma and alpha
param_space = [Real(1.0, 5.0, name='gamma'), Real(0.1, 0.5, name='alpha')]

# Define the objective function
def objective_function(params):
    gamma, alpha = params
    mean_acc, _ = cross_validate_model(x_train, y_train, optimizer='focal', gamma=gamma, alpha=alpha)
    return -mean_acc

# Perform Bayesian Optimization
opt_result = gp_minimize(objective_function, param_space, n_calls=10, random_state=42)

# Extract optimal parameters
best_gamma, best_alpha = opt_result.x
print(f"Optimal gamma: {best_gamma}")
print(f"Optimal alpha: {best_alpha}")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regulariz

Optimal gamma: 1.225646316108401
Optimal alpha: 0.38879950890673


In [ ]:
# Train the model with Tuned AFL
tuned_model = create_model(gamma=best_gamma, alpha=best_alpha)

# Train for 5 epochs
history_tuned = tuned_model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_test, y_test), verbose=1)

# Evaluate on test data
tuned_loss, tuned_acc = tuned_model.evaluate(x_test, y_test)
print(f"Tuned AFL Test Accuracy: {tuned_acc:.4f}")

mean_acc, std_acc = cross_validate_model(x_train, y_train, gamma=best_gamma, alpha=best_alpha)

print(f"Tuned AFL Accuracy: {mean_acc*100:.2f}% ± {std_acc*100:.2f}% ")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 75s 92ms/step - accuracy: 0.3849 - loss: 0.5319 - val_accuracy: 0.5732 - val_loss: 0.3547
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 73s 93ms/step - accuracy: 0.5887 - loss: 0.3346 - val_accuracy: 0.6292 - val_loss: 0.3023
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 79s 89ms/step - accuracy: 0.6456 - loss: 0.2858 - val_accuracy: 0.6505 - val_loss: 0.2788
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 85s 93ms/step - accuracy: 0.6821 - loss: 0.2528 - val_accuracy: 0.6707 - val_loss: 0.2589
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 82s 93ms/step - accuracy: 0.7113 - loss: 0.2226 - val_accuracy: 0.6742 - val_loss: 0.2570
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.6805 - loss: 0.2521
Tuned AFL Test Accuracy: 0.6742


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Tuned AFL Accuracy: 66.40% ± 0.80% 
